## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import inflection
import math

## Compreendendo os dados

- id: um id que representa uma duplicata (Store, Date) dentro do conjunto de teste  

- store: um ID exclusivo para cada loja  

- sales: o volume de negócios para um determinado dia (isto é o que você está prevendo) 

- customers: o número de clientes em um determinado dia  

- open: um indicador para saber se a loja estava aberta: 0 = fechado, 1 = aberto  

- state_holiday: indica um feriado estadual. Normalmente todas as lojas, com poucas exceções, fecham nos feriados estaduais. Observe que todas as escolas fecham nos feriados e fins de semana. a = feriado, b = feriado da Páscoa, c = Natal, 0 = nenhum

- school_holiday: indica se o (Store, Date) foi afetado pelo fechamento das escolas públicas

- store_type: diferencia entre 4 modelos de loja diferentes: a, b, c, d

- assortment: descreve um nível de sortimento: a = básico, b = extra, c = estendido

- competition_distance: distância em metros até a loja concorrente mais próxima

- competition_open_since[Month/Year]: fornece o ano e o mês aproximados da hora em que o concorrente mais próximo foi aberto

- promo: indica se uma loja está realizando uma promoção naquele dia

- promo2: Promo2 é uma promoção contínua e consecutiva para algumas lojas: 0 = loja não está participando, 1 = loja está participando

- promo2_since[Year/Week]: descreve o ano e a semana do calendário em que a loja começou a participar do Promo2

- promoInterval: descreve os intervalos consecutivos em que o Promo2 é iniciado, nomeando os meses em que a promoção é reiniciada. Por exemplo, "fevereiro, maio, agosto, novembro" significa que cada rodada começa em fevereiro, maio, agosto, novembro de qualquer ano para essa loja

In [33]:
#obs: organizar isso depois ;)
#coluna 'promo' não está no dataset

## Carregamento dos dados

In [2]:
df_sales_raw = pd.read_csv('data/train.csv', low_memory=False)
df_store_raw = pd.read_csv('data/store.csv', low_memory=False)

# merge
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')

## Funções adicionais

## Descrição dos dados

### 1. Renomeando colunas

In [3]:
df1 = df_raw.copy()

In [4]:
# nomes das colunas antigas
df1.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [5]:
cols_originais = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday',
                  'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
                  'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']


# criar função para modificar os nomes
def snakecase(x):
    return inflection.underscore(x)


# salvar colunas novas e aplicar função anterior utilizando o map
cols_novas = list(map(snakecase, cols_originais))

# renomear
df1.columns = cols_novas

In [6]:
# checar modificação nos nomes das colunas

df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

### 2. Verificando a dimensão dos dados

In [7]:
print(f'Linhas: {df1.shape[0]}')
print(f'Colunas: {df1.shape[1]}')

Linhas: 1017209
Colunas: 18


### 3. Verificando os tipos de dados

In [8]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [9]:
# coluna 'date' de object para datetime
df1['date'] = pd.to_datetime(df1['date'])

In [10]:
# checando modificações
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

### 4. Checando dados nulos (NA)

In [11]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

### 5. Substituindo os dados nulos

##### comptetition_distance:
- distância em metros do competidor mais próximo
- sugestão: substituir os dados nulos por um número acima do valor máximo de distância, pois ou não há competidor ou está longe o suficiente para não ser considerado um

##### competition_open_since_month:
- mês que o competidor mais próximo foi aberto
- sugestão: substituir os dados nulos do mês de abertura pelo mês da data de venda registrada (coluna 'date') na linha

##### competition_open_since_year:
- ano que o competidor maix pŕoximo foi aberto
- sugestão: substituir os dados nulos do ano de abertura pelo ano da data de venda registrada (coluna 'date') na linha

#### promo2:
- continuação de uma promoção para algumas lojas: 0 -  não está participando; 1- está participando

#### promo2_since_week:
- semana em que a loja começou a participar da promo2

#### promo2_since_year:
- ano em que a loja começou a participar da promo2

##### promo_interval:
- meses em que a promo2 ficou ativa

In [25]:
# encontrar o valor máximo para utilizar de parâmetro para a 'competition_distance'
df1['competition_distance'].max()

200000.0

In [14]:
# competition_distance

df1['competition_distance'] = df1['competition_distance'].apply(
    lambda x: 200000.0 if math.isnan(x) else x)

In [24]:
# competition_open_since_month

df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(
    x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

In [16]:
# competition_open_since_year

df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(
    x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

In [19]:
# promo2_since_week

df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(
    x['promo2_since_week']) else x['promo2_since_week'], axis=1)

In [20]:
# promo2_since_year

df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(
    x['promo2_since_year']) else x['promo2_since_year'], axis=1)

In [18]:
# promo_interval

month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
             7: 'Jul', 8: 'Aug', 9: 'Sept', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

df1['promo_interval'].fillna(0, inplace=True)

df1['month_map'] = df1['date'].dt.month.map(month_map)

df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(
    lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

In [26]:
df1.sample(5).T

,298326,574134,642506,708793,12729
store,291,695,1052,439,465
day_of_week,7,7,2,5,1
date,2014-10-26 00:00:00,2014-02-02 00:00:00,2013-12-03 00:00:00,2013-10-04 00:00:00,2015-07-20 00:00:00
sales,0,0,10699,7179,6872
customers,0,0,973,1011,569
open,0,0,1,1,1
promo,0,0,1,0,0
state_holiday,0,0,0,0,0
school_holiday,0,0,0,1,0
store_type,d,a,a,a,d


In [23]:
# checando o tratamento de NA
df1.isna().sum()

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
month_map                       0
is_promo                        0
dtype: int64

### 6. Etapa 

### 7. Etapa 

### 8. Etapa 

### +